# Deep Networks using TF API

In [1]:
from sklearn.datasets import load_wine

In [2]:
wine_data = load_wine()

In [3]:
type(wine_data)

sklearn.utils.Bunch

In [4]:
wine_data.keys()

dict_keys(['data', 'target_names', 'feature_names', 'DESCR', 'target'])

In [5]:
print(wine_data['DESCR'])

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [6]:
feat_data = wine_data['data']

In [7]:
labels = wine_data['target']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(feat_data, labels, test_size = 0.3, random_state = 101)

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaled_x_train = scaler.fit_transform(X_train)

In [13]:
scaled_x_test = scaler.transform(X_test)

In [14]:
import tensorflow as tf

In [31]:
from tensorflow import estimator

In [30]:
X_train.shape

(124, 13)

## Estimator API 
_(recall section [davizuku] 08-Classification-Exercise)_

In [18]:
feat_cols = [ tf.feature_column.numeric_column('x', shape=[13]) ]

In [19]:
deep_model = estimator.DNNClassifier(hidden_units=[13, 13, 13], 
                                     feature_columns=feat_cols, 
                                     n_classes=3,
                                     optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01)
                                    )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_model_dir': '/var/folders/7f/kl6zxp_50ddcw8xr42yvrpfm0000gn/T/tmplx87rjvs', '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_session_config': None, '_save_summary_steps': 100, '_tf_random_seed': 1}


In [20]:
input_fn = estimator.inputs.numpy_input_fn(x={'x': scaled_x_train}, 
                                           y = y_train,
                                           shuffle = True, 
                                           batch_size = 10, 
                                           num_epochs = 5
                                          )

In [21]:
deep_model.train(input_fn, steps=500)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/7f/kl6zxp_50ddcw8xr42yvrpfm0000gn/T/tmplx87rjvs/model.ckpt.
INFO:tensorflow:step = 1, loss = 10.2677
INFO:tensorflow:Saving checkpoints for 62 into /var/folders/7f/kl6zxp_50ddcw8xr42yvrpfm0000gn/T/tmplx87rjvs/model.ckpt.
INFO:tensorflow:Loss for final step: 3.06751.


In [22]:
input_fn_eval = estimator.inputs.numpy_input_fn(x = {'x': scaled_x_test},
                                                shuffle = False)

In [23]:
preds = list(deep_model.predict(input_fn_eval))

INFO:tensorflow:Restoring parameters from /var/folders/7f/kl6zxp_50ddcw8xr42yvrpfm0000gn/T/tmplx87rjvs/model.ckpt-62


In [25]:
predictions = [ p['class_ids'][0] for p in preds ]

In [27]:
from sklearn.metrics import confusion_matrix,classification_report

In [29]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       1.00      0.32      0.48        19
          1       0.61      0.91      0.73        22
          2       0.87      1.00      0.93        13

avg / total       0.81      0.72      0.69        54



## Keras API

(https://keras.io/)

In [32]:
import tensorflow as tf

In [33]:
from tensorflow.contrib.keras import models

In [34]:
dnn_keras_model = models.Sequential()

In [35]:
from tensorflow.contrib.keras import layers

In [36]:
dnn_keras_model.add(layers.Dense(units=13, input_dim=13, activation='relu'))

In [37]:
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))

In [38]:
dnn_keras_model.add(layers.Dense(units=3, activation='softmax'))

In [39]:
from tensorflow.contrib.keras import losses, optimizers, metrics, activations

In [41]:
dnn_keras_model.compile(optimizer='adam', 
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy']
                       )

In [42]:
dnn_keras_model.fit(scaled_x_train, y_train, epochs=50)

Epoch 1/50
124/124 [==============================] - 0s - loss: 1.0787 - acc: 0.3548     
Epoch 2/50
124/124 [==============================] - 0s - loss: 1.0658 - acc: 0.6210     
Epoch 3/50
124/124 [==============================] - 0s - loss: 1.0516 - acc: 0.7903     
Epoch 4/50
124/124 [==============================] - 0s - loss: 1.0384 - acc: 0.8629     
Epoch 5/50
124/124 [==============================] - 0s - loss: 1.0252 - acc: 0.9113     
Epoch 6/50
124/124 [==============================] - 0s - loss: 1.0115 - acc: 0.9355     
Epoch 7/50
124/124 [==============================] - 0s - loss: 0.9976 - acc: 0.9516     
Epoch 8/50
124/124 [==============================] - 0s - loss: 0.9824 - acc: 0.9597     
Epoch 9/50
124/124 [==============================] - 0s - loss: 0.9659 - acc: 0.9597     
Epoch 10/50
124/124 [==============================] - 0s - loss: 0.9480 - acc: 0.9435     
Epoch 11/50
124/124 [==============================] - 0s - loss: 0.9289 - acc: 0.9516   

In [43]:
predictions = dnn_keras_model.predict_classes(scaled_x_test)

32/54 [================>.............] - ETA: 0s

In [44]:
from sklearn.metrics import classification_report

In [45]:
print(classification_report(predictions, y_test))

             precision    recall  f1-score   support

          0       0.95      0.90      0.92        20
          1       0.86      0.95      0.90        20
          2       1.00      0.93      0.96        14

avg / total       0.93      0.93      0.93        54



## Layers API

(https://www.tensorflow.org/api_guides/python/contrib.layers)

In [46]:
import tensorflow as tf

In [47]:
import pandas as pd

In [50]:
onehot_y_train = pd.get_dummies(y_train).as_matrix()

In [52]:
onehot_y_test = pd.get_dummies(y_test).as_matrix()

In [53]:
num_feat = 13
num_hidden1 = 13
num_hidden2 = 13
num_outputs = 3

In [54]:
learning_rate = 0.01

In [55]:
from tensorflow.contrib.layers import fully_connected

In [56]:
X = tf.placeholder(tf.float32, shape=[None, num_feat])
y_true = tf.placeholder(tf.float32, shape=[None, 3])

In [57]:
actf = tf.nn.relu

In [58]:
hidden1 = fully_connected(X, num_hidden1, activation_fn=actf)
hidden2 = fully_connected(hidden1, num_hidden2, activation_fn=actf)
output = fully_connected(hidden2, num_outputs)

In [59]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

In [60]:
optimizer = tf.train.AdamOptimizer(learning_rate)

In [61]:
train = optimizer.minimize(loss)

In [62]:
init = tf.global_variables_initializer()

In [63]:
training_steps = 1000

In [65]:
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        
        sess.run(train, feed_dict={X: scaled_x_train, y_true: onehot_y_train})
        
    logits = output.eval(feed_dict={X: scaled_x_test})
    
    preds = tf.argmax(logits, axis = 1)
    
    results = preds.eval()

In [66]:
from sklearn.metrics import classification_report

In [67]:
print(classification_report(results, y_test))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        19
          1       0.95      1.00      0.98        21
          2       1.00      0.93      0.96        14

avg / total       0.98      0.98      0.98        54

